In [39]:
import torch
import torchphysics as tp
import time
X = tp.spaces.R2("x")
T = tp.spaces.R1("t")
U = tp.spaces.R1("u")

In [40]:
def grad_fn(u, x, t):
    lap = tp.utils.laplacian(u, x)
    grad = tp.utils.grad(u, x)
    grad_t = tp.utils.grad(u, t)
    return lap + grad[:, :1]**2 - grad_t

grad_compiled = torch.compile(grad_fn)

In [44]:
import torch._dynamo
torch._dynamo.reset()


batch = 500000

model = tp.models.FCN(X*T, U, hidden=(50, 50, 50, 50, 50))
model = model.to("cuda:0")
values = torch.rand((batch, 2), requires_grad=True, device="cuda:0")
values_t = torch.rand((batch, 1), requires_grad=True, device="cuda:0")
input_value = tp.spaces.Points(torch.column_stack((values, values_t)), X*T)
output_value = model(input_value).as_tensor
print("Normal speed")
start_time = time.time()
grad_fn(output_value, values, values_t)
print("Took:", time.time() - start_time)
print("Compiled speed")
start_time = time.time()
grad_compiled(output_value, values, values_t)
print("Took:", time.time() - start_time)

Normal speed
Took: 0.055464982986450195
Compiled speed


BackendCompilerFailed: backend='inductor' raised:
RuntimeError: Cannot find a working triton installation. More information on installing Triton can be found at https://github.com/openai/triton

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True
